In [25]:
import pandas as pd
from IPython.display import display
import matplotlib.pyplot as plt
import seaborn as sns

data = pd.read_csv('Online-eCommerce.csv')
data = data.dropna()

In [26]:
import pandas as pd

# Завантаження даних
data = pd.read_csv('Online-eCommerce.csv')
data = data.dropna()

# Перетворення колонки 'Order_Date' у формат datetime
data['Order_Date'] = pd.to_datetime(data['Order_Date'], errors='coerce')

# Видаляємо некоректні дати
data = data.dropna(subset=['Order_Date'])

# Додаємо колонку з місяцем і роком для групування
data['Order_Month'] = data['Order_Date'].dt.to_period('M')  # Формат YYYY-MM

# Групування за покупцем і місяцем з підрахунком кількості куплених товарів у кожній категорії
transactions = data.groupby(['Customer_Name', 'Order_Month', 'Category']).size().unstack(fill_value=0)

# Перетворюємо кількості на бінарний формат (1 - товар куплено, 0 - не куплено)
transactions = transactions.applymap(lambda x: 1 if x > 0 else 0)

# Переглядаємо результат
display(transactions.head())


C:\Users\911\AppData\Local\Temp\ipykernel_5876\2435410597.py:20: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  transactions = transactions.applymap(lambda x: 1 if x > 0 else 0)


Category                   CPU  Cabinet  Computer Case  Graphic Card  HDD  \
Customer_Name Order_Month                                                   
Adhir Samal   2020-03        0        0              0             1    0   
              2020-05        0        0              0             0    0   
              2020-07        0        0              0             0    0   
              2020-08        0        0              0             0    0   
              2020-09        0        0              0             0    0   

Category                   Keyboard  Monitor  MotherBoard  Motherboard  Mouse  \
Customer_Name Order_Month                                                       
Adhir Samal   2020-03             0        1            0            0      0   
              2020-05             0        1            0            0      0   
              2020-07             0        1            0            0      0   
              2020-08             0        0            0            1      0   
              2020-09             0        0            0            0      1   

Category                   NIC  Printer  RAM  SSD  
Customer_Name Order_Month                          
Adhir Samal   2020-03        0        0    0    1  
              2020-05        0        0    0    0  
              2020-07        0        0    0    1  
              2020-08        0        0    0    0  
              2020-09        0        0    0    0

In [ ]:
from mlxtend.frequent_patterns import fpgrowth, association_rules

frequent_itemsets = fpgrowth(transactions, min_support=0.05, use_colnames=True)

rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)

top_rules = rules.sort_values(by='lift', ascending=False).head(10)
display(top_rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']])


C:\Users\911\anaconda3\envs\myenv\Lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:109: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


,antecedents,consequents,support,confidence,lift
15,(RAM),(CPU),0.053797,0.443478,2.309986
14,(CPU),(RAM),0.053797,0.280220,2.309986
4,(SSD),(Graphic Card),0.058017,0.343750,2.143914
5,(Graphic Card),(SSD),0.058017,0.361842,2.143914
0,(Monitor),(SSD),0.084388,0.347826,2.060870
1,(SSD),(Monitor),0.084388,0.500000,2.060870
11,(HDD),(CPU),0.062236,0.373418,1.945055
10,(CPU),(HDD),0.062236,0.324176,1.945055
6,(Keyboard),(Monitor),0.073840,0.443038,1.826087
7,(Monitor),(Keyboard),0.073840,0.304348,1.826087
